In [12]:
# imports
from tracr.rasp import rasp
from tracr.compiler import compiling
from tracr.compiler import lib

import jax
import numpy as np
import matplotlib.pyplot as plt
jax.config.update('jax_default_matmul_precision', 'float32')

from typing import List, Sequence

In [14]:
#@title Plotting functions
def tidy_label(label, value_width=5):
  if ':' in label:
    label, value = label.split(':')
  else:
    value = ''
  return label + f":{value:>{value_width}}"


def add_residual_ticks(model, value_width=5, x=False, y=True):
  if y:
    plt.yticks(
            np.arange(len(model.residual_labels))+0.5, 
            [tidy_label(l, value_width=value_width)
              for l in model.residual_labels], 
            family='monospace',
            fontsize=20,
    )
  if x:
    plt.xticks(
            np.arange(len(model.residual_labels))+0.5, 
            [tidy_label(l, value_width=value_width)
              for l in model.residual_labels], 
            family='monospace',
            rotation=90,
            fontsize=20,
    )


def plot_computation_trace(model,
                           input_labels,
                           residuals_or_outputs,
                           add_input_layer=False,
                           figsize=(12, 9)):
  fig, axes = plt.subplots(nrows=1, ncols=len(residuals_or_outputs), figsize=figsize, sharey=True)
  value_width = max(map(len, map(str, input_labels))) + 1

  for i, (layer, ax) in enumerate(zip(residuals_or_outputs, axes)):
    plt.sca(ax)
    plt.pcolormesh(layer[0].T, vmin=0, vmax=1)
    if i == 0:
      add_residual_ticks(model, value_width=value_width)
    plt.xticks(
        np.arange(len(input_labels))+0.5,
        input_labels,
        rotation=90,
        fontsize=20,
    )
    if add_input_layer and i == 0:
      title = 'Input'
    else:
      layer_no = i - 1 if add_input_layer else i
      layer_type = 'Attn' if layer_no % 2 == 0 else 'MLP'
      title = f'{layer_type} {layer_no // 2 + 1}'
    plt.title(title, fontsize=20)


def plot_residuals_and_input(model, inputs, figsize=(12, 9)):
  """Applies model to inputs, and plots the residual stream at each layer."""
  model_out = model.apply(inputs)
  residuals = np.concatenate([model_out.input_embeddings[None, ...],
                              model_out.residuals], axis=0)
  plot_computation_trace(
      model=model,
      input_labels=inputs,
      residuals_or_outputs=residuals,
      add_input_layer=True,
      figsize=figsize)


def plot_layer_outputs(model, inputs, figsize=(12, 9)):
  """Applies model to inputs, and plots the outputs of each layer."""
  model_out = model.apply(inputs)
  plot_computation_trace(
      model=model,
      input_labels=inputs,
      residuals_or_outputs=model_out.layer_outputs,
      add_input_layer=False,
      figsize=figsize)


In [23]:
# compute average of a column
def average_numerical_column() -> rasp.SOp:
    square = rasp.Map(lambda x: x**2, rasp.tokens)
    all_selector = rasp.Select(rasp.tokens, square, rasp.Comparison.LT)
    identity_numerical = rasp.numerical(rasp.Map(lambda x: x, rasp.tokens))
    return rasp.numerical(rasp.Aggregate(all_selector, identity_numerical, default=0))
    

average = average_numerical_column()
average([31, 22, 22, 25, 28])

[25.6, 25.6, 25.6, 25.6, 25.6]

In [24]:
# I need to add numerical values
bos = "BOS"
model = compiling.compile_rasp_to_model(
    program=average,
    vocab={5, 10, 15},
    max_seq_len=5,
    compiler_bos=bos,
)

NotImplementedError: ('Attention patterns can currently only average binary variables. Not:', {10, 5, 15})

In [13]:
out = model.apply([bos, 5, 10, 15])
out.decoded

['BOS', 5, 5, 5]

In [27]:
vocab = {"(", ")", "{", "}"}
program = lib.make_shuffle_dyck(pairs=["()", "{}"])
program_name = 'make_shuffle_dyck'
max_seq_len = 10

print(f"Compiling...")
print(f"   Program: {program_name}")
print(f"   Input vocabulary: {vocab}")
print(f"   Context size: {max_seq_len}")

assembled_model = compiling.compile_rasp_to_model(
      program=program,
      vocab=vocab,
      max_seq_len=max_seq_len,
      causal=False,
      compiler_bos="bos",
      compiler_pad="pad",
      mlp_exactness=100)

An NVIDIA GPU may be present on this machine, but a CUDA-enabled jaxlib is not installed. Falling back to cpu.


Compiling...
   Program: make_shuffle_dyck
   Input vocabulary: {'(', ')', '{', '}'}
   Context size: 10


In [29]:
assembled_model.apply(["bos", '(', '{', ')', '}']).decoded

['bos', False, False, False, False]

In [ ]:
assembled_model = compiling.compile_rasp_to_model(
      program=program,
      vocab=vocab,
      max_seq_len=max_seq_len,
      causal=False,
      compiler_bos="bos",
      compiler_pad="pad",
      mlp_exactness=100)

In [ ]:
salary = [62000, 69000, 80000, 60000, 75000]

def select_employee_where_salary_above_x_below_y(x: rasp.SOp, y: rasp.SOp) -> rasp.SOp:
    # above
    bools_above = rasp.numerical(x)
    # below
    bools_below = rasp.numerical(y)
    rasp.Select(bools_above, bools_above, )

In [44]:
def select_employee_where_salary_above_x_below_y2(x: int, y: int) -> rasp.SOp:

    return rasp.Map(lambda z: x < int(z) < y, rasp.tokens)

select = select_employee_where_salary_above_x_below_y2(65000, 70000)

select([62000, 69000, 80000, 60000, 75000])

[0, 1, 2, 3, 4]

In [42]:
vocab = {'62000', '69000', '80000', '60000', '75000'}

assembled_model = compiling.compile_rasp_to_model(
      program=select,
      vocab=vocab,
      max_seq_len=max_seq_len,
      causal=False,
      compiler_bos="bos",
      compiler_pad="pad",
      mlp_exactness=100)

In [35]:
assembled_model.apply(["bos", 62000, 69000, 80000, 60000, 75000]).decoded

['bos', False, True, False, False, False]

In [ ]:
# model it by combining the values
def select_employee_where_salary_above_x_below_y_with_bonus(x: int, y: int) -> rasp.SOp:
    return rasp.Map(lambda z: x < z < y, rasp.tokens)
    

In [49]:
def shift_by(offset: int, /, sop: rasp.SOp) -> rasp.SOp:
  """Returns the sop, shifted by `offset`, None-padded."""
  select_off_by_offset = rasp.Select(rasp.indices, rasp.indices,
                                     lambda k, q: q == k + offset)
  out = rasp.Aggregate(select_off_by_offset, sop, default=None)
  return out.named(f"shift_by({offset})")

shift = shift_by(2, rasp.tokens)

def detect_pattern(sop: rasp.SOp, pattern: Sequence[rasp.Value]) -> rasp.SOp:
  """Returns an SOp which is True at the final element of the pattern.

  The first len(pattern) - 1 elements of the output SOp are None-padded.

  detect_pattern(tokens, "abc")("abcabc") == [None, None, T, F, F, T]

  Args:
    sop: the SOp in which to look for patterns.
    pattern: a sequence of values to look for.

  Returns:
    a sop which detects the pattern.
  """

  if len(pattern) < 1:
    raise ValueError(f"Length of `pattern` must be at least 1. Got {pattern}")

  # detectors[i] will be a boolean-valued SOp which is true at position j iff
  # the i'th (from the end) element of the pattern was detected at position j-i.
  detectors = []
  for i, element in enumerate(reversed(pattern)):
    detector = sop == element
    if i != 0:
      detector = shift_by(i, detector)
    detectors.append(detector)

  # All that's left is to take the AND over all detectors.
  pattern_detected = detectors.pop()
  while detectors:
    pattern_detected = pattern_detected & detectors.pop()

  return pattern_detected.named(f"detect_pattern({pattern})")

# Modeling JOIN

In [4]:
def join_and_calculate_bonuses() -> rasp.SOp:
    BONUS = 5000

    select_by_foreign_keys = rasp.Select(rasp.tokens, rasp.indices, rasp.Comparison.EQ)
    bonus_counts = rasp.SelectorWidth(select_by_foreign_keys)

    salaries_with_bonuses = rasp.SequenceMap(lambda x, y: x + y * BONUS, rasp.tokens, bonus_counts)

    return salaries_with_bonuses

join = join_and_calculate_bonuses()

join([62000, 69000, 80000, 60000, 75000, 2, 1, 2, 0, 4])

[67000, 74000, 90000, 60000, 80000, 2, 1, 2, 0, 4]

In [6]:
vocab = {62000, 69000, 80000, 60000, 75000, 2, 1, 2, 0, 4}
max_seq_len = 10

assembled_model = compiling.compile_rasp_to_model(
      program=join,
      vocab=vocab,
      max_seq_len=max_seq_len,
      causal=False,
      compiler_bos="bos",
      compiler_pad="pad",
      mlp_exactness=100)

An NVIDIA GPU may be present on this machine, but a CUDA-enabled jaxlib is not installed. Falling back to cpu.


In [7]:
assembled_model.apply(["bos", 62000, 69000, 80000, 60000, 75000, 2, 1, 2, 0, 4]).decoded

['bos', 67000, 74000, 90000, 60000, 80000, 2, 1, 2, 0, 4]

In [16]:
#@title Plot residual stream
plot_residuals_and_input(
  model=assembled_model,
  inputs=["bos", 62000, 69000, 80000, 60000, 75000, 2, 1, 2, 0, 4],
  figsize=(100, 90)
)

In [83]:
join2 = join_and_calculate_bonuses2()
join2([0, 0, 0, 0, 0, 2, 1, 2, 0, 4])

[2, 1, 2, 0, 4, None, None, None, None, None]